#Запуск

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install vk_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
import json
import vk_api
import time
import numpy as np
import pandas as pd
from collections import Counter
import statistics as st

In [ ]:
def captcha_handler(captcha):

    key = input("Enter captcha code {0}: ".format(captcha.get_url())).strip()

    # Повтор запроса с капчей
    return captcha.try_again(key)

#чтение файла с логином и паролем
f = open("login.txt", "r")

#открытие сесси вк
vk_session = vk_api.VkApi(f.readline(), f.readline(), captcha_handler=captcha_handler)
vk_session.auth(token_only=True)
vk = vk_session.get_api()

#Тест

In [ ]:
getFirst = vk.users.get(user_ids = '122257440,218911746', fields="counters")

getFirst

# print(getFirst[0]['bdate'])
# print(getFirst[0]['university'])
# print(getFirst[0]['personal']['langs'])
# print(getFirst[0]['city'])

In [ ]:
getFirst = vk.friends.get(user_id = '218911746')
getFirst['count']

119

In [ ]:
getFirst = vk.users.getSubscriptions(user_id = '218911746', extend=0)
getFirst["groups"]["count"]

# string = ""
# for i in range(min(20,len(getFirst["groups"]["items"]))):   
#   string += str(getFirst["groups"]["items"][i]) + ","

# string = string[:-1]
# print(string) 

28

In [ ]:
getFirstCom = vk.groups.getById(group_id = '139105204', fields='activity')
getFirstCom[0]["activity"]

'Юмор'

# Обработка данных

In [ ]:
firstCom = 'miem_hse'
secondCom = 'miemos'
count = '1000'

In [ ]:
#сбор id подписчиков и их друзей
def collectingData(firstCom, secondCom, count, vk):


    #сбор данных об участниках групп
    getFirstCom = vk.groups.getMembers(group_id = firstCom, count = count, offset = '2000')
    getSecondCom = vk.groups.getMembers(group_id = secondCom, count = count, offset = '100')

    #поиск повторящихся пользователей пабликов
    getCommon, getFirstCom, getSecondCom = searchCommon(getFirstCom["items"], getSecondCom["items"])
        
    dataFriends = {
        firstCom : {},
        secondCom : {}, 
        "commonCom" : {}
    }

    countFirst = 0
    countSecond = 0
    countCommon = 0
   
    time.sleep(3)

    # данные о друзьях 1 паблика
    for i in range(len(getFirstCom)):           
        try:
            getFriends = vk.friends.get(user_id = str(getFirstCom[i]))
            g = {str(getFirstCom[i]): getFriends['items']}
            dataFriends[firstCom].update(g)
        except:
            countFirst += 1
    
    time.sleep(3)

    # данные о друзьях 2 паблика
    for i in range(len(getSecondCom)):           
        try:
            getFriends = vk.friends.get(user_id = str(getSecondCom[i]))
            g = {str(getSecondCom[i]): getFriends['items']}
            dataFriends[secondCom].update(g)
        except:
            countSecond += 1
   
    time.sleep(3)

    # данные о друзьях общего паблика
    for i in range(len(getCommon)):           
        try:
            getFriends = vk.friends.get(user_id = str(getCommon[i]))
            g = {str(getCommon[i]): getFriends['items']}
            dataFriends["commonCom"].update(g)
        except:
            countCommon += 1

    with open('data_file.json', 'w') as outfile:
        json.dump(dataFriends, outfile)
    
    return countFirst, countSecond, countCommon

In [ ]:
def searchCommon(a, b):
  res1 = []
  res2 = []
  common = []
  for i in range(len(a)):
      try: 
          b.index(a[i])
          common.append(a[i])
      except:
          res1.append(a[i])  

  for i in range(len(b)):
      try: 
          a.index(b[i])
      except:
          res2.append(b[i])  
   
  return common, res1, res2

In [ ]:
#запуск сбора данных
countFirst, countSecond, countCommon = collectingData(firstCom, secondCom, count, vk)
print(countFirst, countSecond, countCommon)

In [ ]:
# Матрица смежности
def processingData(firstCom, secondCom):

    #чтение файла json
    with open("data_file(9).json", "r") as infile:
        data = infile.read()

    dataFriends = json.loads(data)

    #передеслакация 
    dl = []
    for i in dataFriends[firstCom]:
        if vk.groups.isMember(group_id = secondCom, user_id  = i ):
            dataFriends["commonCom"][i] = dataFriends[firstCom][i]
            dl.append(i)
    for i in dl:
        dataFriends[firstCom].pop(i)
    dl = []
    for i in dataFriends[secondCom]:
        if vk.groups.isMember(group_id = firstCom, user_id  = i ):
            dataFriends["commonCom"][i] = dataFriends[secondCom][i]
            dl.append(i)
    for i in dl:
        dataFriends[secondCom].pop(i)

    #определение длины каждого словаря
    lenFirst = len(dataFriends[firstCom])
    lenSecond = len(dataFriends[secondCom])
    lenCommon = len(dataFriends["commonCom"])
    lenTotal = lenFirst + lenSecond + lenCommon + 2
    rv = {}#rv - вектор смежности

    #ids
    idsFirst = [*dataFriends[firstCom]]
    idsSecond = [*dataFriends[secondCom]]
    idsCommon = [*dataFriends["commonCom"]]
    idsTotal = set([int(i) for i in idsFirst+idsSecond+idsCommon])

    #соединение с сообществом
    rv[firstCom] = [int(i) for i in idsFirst+idsCommon]
    rv[secondCom] = [int(i) for i in idsSecond+idsCommon]
    
    #соединение друзей
    ttl = dataFriends[firstCom]|dataFriends[secondCom]|dataFriends["commonCom"]
    for i in ttl:
        rv[int(i)] = list(set(ttl[i]) & idsTotal)

    #преобразование в матрицу
    ks = list(rv.keys())
    R = {}
    for id in ks:
        R[str(id)] = [str(id)] + [int(ks[j] in rv[id]) for j in range(len(ks))]
    R = [["X"]+list(R.keys())]+list(R.values())

    return R ,lenTotal, lenFirst, lenCommon

In [ ]:
R, lenTotal, lenFirst, lenCommon = processingData(firstCom, secondCom)
df = pd.DataFrame(R)

In [ ]:
df.to_excel(r"drive/MyDrive/Masters_dissertation/output/matrix2.xlsx", index=False)
df.to_excel(r"matrix2.xlsx", index=False)

In [ ]:
print(lenTotal)

1197


#Статистика

In [ ]:
R = []
excel_data = pd.read_excel('matrix.xlsx')
data = pd.DataFrame(excel_data)

In [ ]:
lenTotal = 32

In [ ]:
index_name = []
for i in range(lenTotal):
  index_name.append(data.iloc[0][i+1])
index_name

In [ ]:
with open("data_with_distribution.json", "r") as infile:
        data_json = infile.read()
data_json= json.loads(data_json)
type(data_json)

dict

In [ ]:
#Статистика для топологии сети
Th = [0, 0]
mem = [1, 1]
disc = [1, 1]

for i in range(len(index_name)):
  for item in data_json:
    if str(index_name[i]) in data_json[item].keys():
      Th.append(data_json[item][str(index_name[i])]["param"]["Th"])
      if data_json[item][str(index_name[i])]["param"]["type"] == "аналитик":
        mem.append(3)
        disc.append(0.7)
      else:
        mem.append(1)
        disc.append(1)

In [ ]:
mem.pop(0)

1

In [ ]:
with open("data_all.json", "r") as infile:
        data_json = infile.read()
data_json= json.loads(data_json)
type(data_json)

#для тестовой выборки
Th = []
mem = []
disc = []

for item in data_json:
  Th.append(data_json[item]["param"]["Th"])
  if data_json[item]["param"]["type"] == "аналитик":
    mem.append(3)
    disc.append(0.7)
  else:
    mem.append(1)
    disc.append(1)

c = Counter(mem)
c

In [ ]:
Th = []
mem = []
disc = []

for pubItem in data_json:
  for item in data_json[pubItem]:
    Th.append(data_json[pubItem][item]["param"]["Th"])
    if data_json[pubItem][item]["param"]["type"] == "аналитик":
      mem.append(3)
      disc.append(0.7)
    else:
      mem.append(1)
      disc.append(1)

In [ ]:
c = Counter(mem)
c

Counter({1: 1080, 3: 110})

In [ ]:
print("mean: ",st.mean(Th))
print("median: ",st.median(Th))
print("median_low: ",st.median_low(Th))
print("median_high: ",st.median_high(Th))
print("median_grouped: ",st.median_grouped(Th))
print("mode: ",st.mode(Th))
print("Отрезок: [",min(Th), ";", max(Th), "]")

mean:  0.8573522799160661
median:  0.6239373991891539
median_low:  0.6157635467980297
median_high:  0.6321112515802781
median_grouped:  0.13211125158027814
mode:  0.23212627669452182
Отрезок: [ 0.1868460388639761 ; 5.1020408163265305 ]


#Словарь со списками, добавление параметров

In [ ]:
with open("data_file(9).json", "r") as infile:
        data = infile.read()

dataFriends = json.loads(data)

In [ ]:
dataAll = {}
f = 0
for bigPub in dataFriends:
    dataAll[bigPub] = {}
    
    for item in dataFriends[bigPub]:
        # time.sleep(3)

        try:
            getUser_sub = vk.users.getSubscriptions(user_id = item, extend=0)
        except:
            continue

        dataAll[bigPub][item] = {}
        dataAll[bigPub][item]["publics"] = {}
        # dataAll[bigPub][item]["info"] = {}

        getUser_info = vk.users.get(user_id = item, fields="bdate, education, personal, city")
        getUser_friend = vk.friends.get(user_id = item)

        dataAll[bigPub][item]["info"] = getUser_info[0]
        dataAll[bigPub][item]["info"]["count_friends"] = getUser_friend["count"]

        grp_ids = ""

        for i in range(min(20,len(getUser_sub["groups"]["items"]))):   
          grp_ids += str(getUser_sub["groups"]["items"][i]) + ","

        grp_ids = grp_ids[:-1]
        
        # try:
        #   getCom = vk.groups.getById(group_ids = grp_ids, fields='activity')
        # except:
        #   getCom = ""

        # for i in range(len(getCom)):
        #   dataAll[bigPub][item]["publics"][getUser_sub["groups"]["items"][i]] = getCom[i]["activity"]

        try:
          getCom = vk.groups.getById(group_ids = grp_ids, fields='activity')
          for i in range(len(getCom)):
            dataAll[bigPub][item]["publics"][getUser_sub["groups"]["items"][i]] = getCom[i]["activity"]
        except:
          getCom = ""
          for i in range(len(getCom)):
              dataAll[bigPub][item]["publics"][getUser_sub["groups"]["items"][i]] = "Сайты"

        dataAll[bigPub][item]["info"]["count_publics"] = getUser_sub["groups"]["count"]

        dataAll[bigPub][item]["friends"] = dataFriends[bigPub][item]

In [ ]:
with open('data_all.json', 'w') as outfile:
    json.dump(dataAll, outfile, ensure_ascii=False)

with open(r"drive/MyDrive/Masters_dissertation/output/data_all.json", 'w') as outfile:
    json.dump(dataAll, outfile, ensure_ascii=False)

##Определение типов
Th = 100/(кол-во друзей х 0,6 + кол-во групп х 0,4)

p = 

Определение аналитика и эмоционального. По кол-ву пабликов, порог 50%.


In [ ]:
import datetime
from datetime import date

In [ ]:
with open("data_all.json", "r") as infile:
        data = infile.read()

dataAll = json.loads(data)

In [ ]:
def qualimetric_indicator(age, langs, city, edu, pub):
  
  qi= 0

  if age < 18: 
    qi += 0.05
  elif age >= 18 and age < 25:
    qi += 0.1
  elif age >= 25 and age < 35:
    qi += 0.15
  elif age >= 35 and age < 50:
    qi += 0.2
  else:
    qi += 0.25

  if langs < 2:
    qi +=0.05
  elif langs == 2:
    qi += 0.1
  elif langs == 3:
    qi += 0.15
  else:
    qi += 0.2

  if city == 4:
    qi +=0.4
  elif city == 3:
    qi +=0.3
  elif city == 2:
    qi +=0.2
  else:
    qi +=0.1

  if edu == 3:
    qi +=0.45
  elif edu == 2:
    qi +=0.3
  else:
    qi +=0.15

  if pub < 20:
    qi +=0.65
  elif pub >=20 and pub < 40:
    qi +=1.3
  elif pub >=40 and pub < 65:
    qi +=1.95
  else:
    qi +=2.6
  
  
  return qi/3.9

In [ ]:
def pre_qualimetric(user_info, getCityBig, getUni_list, getType):
 
  qi = 0
  today = date.today()

  if "bdate" in user_info["info"]:
    d = user_info["info"]["bdate"].split(".")
    if len(d) == 3:
      age = today.year - int(d[2]) - ((today.month, today.day) < (int(d[1]), int(d[0])))
    else:
      age = 18
  else:
    age = 18

  if "langs" in user_info["info"]:
    langs = len(user_info["info"]["langs"])
  else: 
    langs = 0

  if "city" in user_info["info"]:
    if user_info["info"]["city"]["id"] == 1 or user_info["info"]["city"]["id"] == 2:
      city = 4
    elif user_info["info"]["city"]["id"] in getCityBig["items"][0].values():
      city = 3
    else: 
      city = 2
  else:
    city = 1
    
  if "university" in user_info["info"]:
    if user_info["info"]["university"] in getUni_list:
      edu = 3
    else:
      edu = 2
  else:
    edu = 1

  pub = [0,0,0]
  cnt = [0,0,0]

  for i in user_info['publics']:

      try:
          cnt[int(getType[user_info['publics'][i]])]+=1
      except KeyError:
          cnt[1]+=1


  for i in range(3):
    try:
      pub[i] = cnt[i]/sum(cnt)*100 
    except ZeroDivisionError:
      pub[i] = 0

  qi = qualimetric_indicator(age, langs, city, edu, pub[2])

  if qi < 0.8:
    return "эмоциональный"
  else:
    return "аналитик"

In [ ]:
getCityBig = vk.database.getCities(country_id="1")

getUni = []
getUni1 = vk.database.getUniversities(city_id = "1")
getUni2 = vk.database.getUniversities(city_id = "2")

getUni.extend(getUni1["items"])
getUni.extend(getUni2["items"])

getUni_list = []

for i in range(len(getUni)):

  getUni_list.append(getUni[i]["id"])

getUni_list = set(getUni_list)

groups_types = pd.read_excel('drive/MyDrive/Masters_dissertation/input/groups_types.xlsx')

getType = {}
for i in range(len(groups_types["Type"])):
    getType[groups_types["Type"][i]] = groups_types["Score"][i]

In [ ]:
for pubItem in dataAll:
  for item in dataAll[pubItem]:

    dataAll[pubItem][item]["param"] = {}

    try:
      dataAll[pubItem][item]["param"]["Th"] = 100/(0.6*dataAll[pubItem][item]["info"]["count_friends"] \
                                                 + 0.4*dataAll[pubItem][item]["info"]["count_publics"])
    except:
      dataAll[pubItem][item]["param"]["Th"] = 0
    
    dataAll[pubItem][item]["param"]["type"] = pre_qualimetric(dataAll[pubItem][item], getCityBig, getUni_list, getType)

In [ ]:
with open('data_with_distribution.json', 'w') as outfile:
    json.dump(dataAll, outfile, ensure_ascii=False)

# Сбор статистики случайных групп

In [ ]:
getFirstCom = vk.groups.getById(group_id = 'fuck_humor', fields='activity')
getFirstCom[0]["activity"]

'Юмор'

In [ ]:
def collectingData(community, count):
  getFirstCom = vk.groups.getMembers(group_id = community, count = count, offset = '20000')

  dataFriends = {}
  countFirst = 0

  for i in range(len(getFirstCom["items"])):           
    try:
        getFriends = vk.friends.get(user_id = str(getFirstCom["items"][i]))
        g = {str(getFirstCom["items"][i]): getFriends['items']}
        dataFriends.update(g)
    except:
        countFirst += 1

  with open('data_file.json', 'w') as outfile:
        json.dump(dataFriends, outfile)

collectingData("fuck_humor", 200)

In [ ]:
getCityBig = vk.database.getCities(country_id="1")

getUni = []
getUni1 = vk.database.getUniversities(city_id = "1")
getUni2 = vk.database.getUniversities(city_id = "2")

getUni.extend(getUni1["items"])
getUni.extend(getUni2["items"])

getUni_list = []

for i in range(len(getUni)):

  getUni_list.append(getUni[i]["id"])

getUni_list = set(getUni_list)

groups_types = pd.read_excel('drive/MyDrive/Masters_dissertation/input/groups_types.xlsx')

getType = {}
for i in range(len(groups_types["Type"])):
    getType[groups_types["Type"][i]] = groups_types["Score"][i]

In [ ]:
with open("data_file.json", "r") as infile:
        data = infile.read()

dataFriends = json.loads(data)

In [ ]:
len(dataFriends)

128

In [ ]:
dataAll = {}
f = 0

dataAll = {}

for item in dataFriends:
  
    try:
        getUser_sub = vk.users.getSubscriptions(user_id = item, extend=0)
    except:
        continue

    dataAll[item] = {}
    dataAll[item]["publics"] = {}

    getUser_info = vk.users.get(user_id = item, fields="bdate, education, personal, city")
    getUser_friend = vk.friends.get(user_id = item)

    dataAll[item]["info"] = getUser_info[0]
    dataAll[item]["info"]["count_friends"] = getUser_friend["count"]

    grp_ids = ""

    for i in range(min(20,len(getUser_sub["groups"]["items"]))):   
      grp_ids += str(getUser_sub["groups"]["items"][i]) + ","

    grp_ids = grp_ids[:-1]

    try:
      getCom = vk.groups.getById(group_ids = grp_ids, fields='activity')
      for i in range(len(getCom)):
        dataAll[item]["publics"][getUser_sub["groups"]["items"][i]] = getCom[i]["activity"]
    except:
     for i in range(len(getCom)):
        dataAll[item]["publics"][getUser_sub["groups"]["items"][i]] = "Сайты"

    dataAll[item]["info"]["count_publics"] = getUser_sub["groups"]["count"]

    dataAll[item]["friends"] = dataFriends[item]

with open('data_all.json', 'w') as outfile:
    json.dump(dataAll, outfile, ensure_ascii=False)

In [ ]:
with open("data_all.json", "r") as infile:
        data = infile.read()

dataAll = json.loads(data)

In [ ]:
for pubItem in dataAll:
  for item in dataAll:

    dataAll[item]["param"] = {}

    dataAll[item]["param"]["Th"] = 100/(0.6*dataAll[item]["info"]["count_friends"] \
                                                 + 0.4*dataAll[item]["info"]["count_publics"])
    dataAll[item]["param"]["type"] = pre_qualimetric(dataAll[item], getCityBig, getUni_list, getType)

In [ ]:
with open('data_all.json', 'w') as outfile:
    json.dump(dataAll, outfile, ensure_ascii=False)

In [ ]:
with open("data_all.json", "r") as infile:
        data_json = infile.read()
data_json= json.loads(data_json)
type(data_json)

#для тестовой выборки
Th = []
mem = []
disc = []

for item in data_json:
  Th.append(data_json[item]["param"]["Th"])
  if data_json[item]["param"]["type"] == "аналитик":
    mem.append(3)
    disc.append(0.7)
  else:
    mem.append(1)
    disc.append(1)

c = Counter(mem)
c

Counter({1: 126, 3: 2})

#Draft

In [ ]:
def processingData(firstCom, secondCom, vk):

    #чтение файла json
    with open("data_1.json", "r") as infile:
        data = infile.read()

    dataFriends = json.loads(data)

    R = []

    #определение длины каждого словаря
    lenFirst = len(dataFriends[firstCom])
    lenSecond = len(dataFriends[secondCom])
    lenCommon = len(dataFriends["commonCom"])
 
    lenTotal = lenFirst + lenSecond + lenCommon + 2

    for i in range(lenTotal):
        e = np.zeros(lenTotal)
        R.append(e)

    print(lenFirst)
    print(lenSecond)
    print(lenCommon)


    print(len(R[0]))
    idsFirst = [*dataFriends[firstCom]]
    idsSecond = [*dataFriends[secondCom]]
    idsCommon = [*dataFriends["commonCom"]]

    #определение связей между сообществами и пользователями
    for i in range(lenTotal):
      if i != 0 and i < lenFirst + 1 + lenCommon:
        R[0][i] = 2

      if (i >= lenFirst + 1) and (i != lenFirst + 1 + lenCommon):
        R[lenFirst + 1 + lenCommon][i] = 2

    # есть ли пользователи 1го сообщества во 2м 
    for i in range(len(idsFirst)):
      if vk.groups.isMember(group_id = secondCom, user_id = idsFirst[i]):
        R[lenFirst + 1 + lenCommon][i+1]=1
    # есть ли пользователи общего сообщества во 2м 
    for i in range(len(idsCommon)):
      if vk.groups.isMember(group_id = secondCom, user_id = idsCommon[i]):
        R[lenFirst + 1 + lenCommon][i+1+lenFirst]=1
    # есть ли пользователи 2го сообщества во 1м 
    for i in range(len(idsSecond)):
      if vk.groups.isMember(group_id = firstCom, user_id = idsSecond[i]):
        R[0][i+1+lenFirst + lenCommon]=1
    # есть ли пользователи общего сообщества во 2м 
    for i in range(len(idsCommon)):
      if vk.groups.isMember(group_id = firstCom, user_id = idsCommon[i]):
        R[0][i+1+lenFirst]=1


    #определение связей между пользователями первого сообщества
    
    for i in range(len(idsFirst)):
      for j in range(i, len(idsFirst)):
        for k in range(len(dataFriends[firstCom][idsFirst[j]])):
          if int(idsFirst[i]) == dataFriends[firstCom][idsFirst[j]][k]:
            R[i+1][j+1] = 1
            R[j+1][i+1] = 1
          
    #определение связей между пользователями второго сообщества
    
    for i in range(len(idsSecond)):
      for j in range(i, len(idsSecond)):
        for k in range(len(dataFriends[secondCom][idsSecond[j]])):
          if int(idsSecond[i]) == dataFriends[secondCom][idsSecond[j]][k]:
            R[i+2 + lenFirst + lenCommon][j+2 + lenFirst + lenCommon] = 1
            R[j+2 + lenFirst + lenCommon][i+2 + lenFirst + lenCommon] = 1

    #определение связей между одинаковыми пользователями    
    for i in range(len(idsCommon)):
      for j in range(i, len(idsCommon)):
        for k in range(len(dataFriends["commonCom"][idsCommon[j]])):
          if int(idsCommon[i]) == dataFriends["commonCom"][idsCommon[j]][k]:
            R[i+1 + lenFirst][j+1 + lenFirst] = 1
            R[j+1 + lenFirst][i+1 + lenFirst] = 1

    #определение связей между первым сообществом и общими
    for i in range(len(idsFirst)):
      for j in range(len(idsCommon)):
        for k in range(len(dataFriends["commonCom"][idsCommon[j]])):
          if int(idsFirst[i]) == dataFriends["commonCom"][idsCommon[j]][k]:
            R[i+1][j+1+lenFirst] = 1
            R[j+1+lenFirst][i+1] = 1

    #определение связей между первым и вторым сообществами
    for i in range(len(idsFirst)): #от 0 до длины F
      for j in range(len(idsSecond)): #от 0 до длины S
        for k in range(len(dataFriends[secondCom][idsSecond[j]])): #от 0 до длины друзей у польз. второго сообщ.
          if int(idsFirst[i]) == dataFriends[secondCom][idsSecond[j]][k]: #если пользователь F равен id другу пользователя S
            R[j+2+lenFirst+lenCommon][i+1] = 1
            R[i+1][j+2+lenFirst+lenCommon] = 1 

    #определение связей между вторым сообществом и общими
    for i in range(len(idsSecond)):
      for j in range(len(idsCommon)):
        for k in range(len(dataFriends["commonCom"][idsCommon[j]])):
          if int(idsSecond[i]) == dataFriends["commonCom"][idsCommon[j]][k]:
            R[i+2+lenFirst+lenCommon][j+1+lenFirst] = 1
            R[j+1+lenFirst][i+2+lenFirst+lenCommon] = 1


    return R, lenTotal, lenFirst, lenCommon

In [ ]:
import random as ran

In [ ]:
n = 32
st = []

for i in range(n):
  g = []
  p = []
  y = []
  p_sum = 0

  h = ran.betavariate(0.1, 0.1)
  while h < 0.1 or h >0.9 and h > 0.6:
      h = ran.betavariate(0.1, 0.1)
  g.append(h)
  g.append(1-h)

  st.append(g) 
for i in range(len(st)):
  print(st[i])

[0.23181948554537862, 0.7681805144546214]
[0.2627321650647504, 0.7372678349352496]
[0.3594784604454999, 0.6405215395545001]
[0.1705609418876453, 0.8294390581123547]
[0.3330058256840025, 0.6669941743159975]
[0.18979126440991362, 0.8102087355900864]
[0.21680366428404166, 0.7831963357159584]
[0.8388149275911996, 0.16118507240880042]
[0.3130948776680445, 0.6869051223319556]
[0.1485099779603098, 0.8514900220396902]
[0.3245991675093053, 0.6754008324906947]
[0.5561441809543343, 0.44385581904566573]
[0.5403794918929864, 0.4596205081070136]
[0.5681164474250254, 0.43188355257497457]
[0.6671752257962454, 0.33282477420375456]
[0.6071808398979864, 0.39281916010201356]
[0.24777745128055045, 0.7522225487194496]
[0.32767774576282094, 0.6723222542371791]
[0.8123378610558465, 0.1876621389441535]
[0.5069577994891717, 0.49304220051082825]
[0.10305654779669746, 0.8969434522033025]
[0.2533770132889029, 0.7466229867110972]
[0.3346765366062229, 0.6653234633937771]
[0.5697416418307206, 0.4302583581692794]
[0.1

In [ ]:
s = "0.7"
s.replace
s1 = float(s.replace(",", "."))
print(s1)

0.7
